In [ ]:
import torch, platform
print("Python:", platform.python_version())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


In [ ]:
REPO = "https://github.com/HenryNVP/anime_recsys.git"
!git clone $REPO

import os, re
repo_name = re.sub(r".*/(.*)\\.git$", r"\\1", REPO)
%cd $repo_name

!pip -q install -r requirements.txt

In [ ]:
# Create data/ and upload anime.csv, rating.csv
import os
os.makedirs("data", exist_ok=True)

from google.colab import files
print("Upload anime.csv and rating.csv")
uploaded = files.upload()
for name in uploaded.keys():
    if name.endswith(".csv"):
        !mv $name data/
!ls -lh data || true

In [ ]:
# Adjust --epochs and --batch_size for speed/quality
!python -m recsys.train --trainer neumf --epochs 5 --batch_size 8192 --neg_k 4

In [ ]:
# Uncomment to train ALS baseline (requires 'implicit')
!python -m recsys.train --trainer als --factors 64 --iterations 10 --reg 0.01

In [ ]:
print("NeuMF (val):")
!python -m recsys.eval --model neumf --split val --k 10

print("\nNeuMF (test):")
!python -m recsys.eval --model neumf --split test --k 10

print("\nALS (val):")
!python -m recsys.eval --model als --split val --k 10 || true

print("\nALS (test):")
!python -m recsys.eval --model als --split test --k 10 || true

In [ ]:
import json, os, matplotlib.pyplot as plt

if os.path.exists("results/metrics.json"):
    with open("results/metrics.json") as f:
        m = json.load(f)
    print(m)
else:
    print("Run eval first.")